## Modelo y metodología para el robo a casa habitación

El modelo que se plantea explica el robo a casa habitación en función de la población de 25 a 34 años, la tasa de desocupación, el PIB estatal y la pobreza.

Las datos provienen de distintas fuentes:

1. La Secretaría de Gobernación, para la variable de robo a casa habitación ($Tasa\_rcasa$); 
2. CONAPO, en el caso de la población de 25 a 34 años ($prop2534$);
3. INEGI, a través de la ENOE, brinda la tasa de desocupación ($desocupa$);
3. El Sistema de Cuentas Nacionales del INEGI porporciona el PIB estatal ($pibe$) y;
4. El CONEVAL presenta la estimación de la pobreza en el año 2016 y 2018. Las cifras de pobreza de los años 2015 y 2017 se estimaron con interpolación por splines ($pobreza$).

Con esta información se conforma una base de datos panel, se siguen las 32 entidades federativas a través del periodo 2015 a 2019. Es decir, se tienen las variables señaladas por cada estado durante 5 años, con excepción de la pobreza, cuya información esta disponible hasta 2018, debido a que la base de datos utilizada para su construcción se produce de manera bianual. Por tal razón, el panel de datos es desbalanceado.

Dada la estructura de la información se utiliza el método de estimación de mínimos cuadrados ordinarios para datos panel con efectos aleatorios. Esta metodología permite eliminar el efecto de variables omitidas que son constantes en el tiempo y que podrían causar inconsistencia en el modelo.

A fin de facilitar la interpretación los resultados de la estimación se realizó la transformación logarítmica de la variable PIB estatal.

Así, el modelo planteado es el siguiente:

$$Tasa\_rcasa_{i,j}=\beta_0+\beta_1*prop2534_{i,j}+\beta_2*desocupa_{i,j}-\beta_3*logpibe_{i,j}-\beta_4*pobreza_{i,j}+u$$

Donde:
    i = entidad
    j = periodo
    Tasa\_rcasa = tasa de robo a casa habitación por cada 100 mil habitantes
    prop2534 = proporción de la población de 25 a 34 años respecto del total de la población
    desocupa = tasa de desocupación
    logpibe = logaritmo natural de PIB estatal
    pobreza = porcentaje de la población en pobreza      

# Estimación del modelo

In [2]:
import pandas as pd
import numpy as np

La estimación de los modelos se realiza a partir de https://bashtage.github.io/linearmodels/panel/examples/examples.html

In [3]:
bd = pd.read_csv('base_final.csv')

In [4]:
year = pd.Categorical(bd["ao"])

In [5]:
data = bd.set_index(["edo","ao"])

In [6]:
data['ao'] = year

In [7]:
print(data.head())

                     clave_ent  homicidio  robocasa  robocalle  robocarro  \
edo            ao                                                           
Aguascalientes 2015          1        211      2525       1754       1995   
               2016          1        233      2459       1353       3110   
               2017          1        258      3327       1949       3624   
               2018          1        283      3355       2216       3309   
               2019          1        269      2817       1780       2399   

                     extorsion  tasa_hom  tasa_rcasa  tasa_rcalle  \
edo            ao                                                   
Aguascalientes 2015         39     15.84      189.59       131.70   
               2016         70     17.19      181.43        99.83   
               2017        121     18.75      241.83       141.66   
               2018        138     20.28      240.36       158.76   
               2019        116     19.00      

In [8]:
data["logpibe"] = np.log(data["pibe"])
data["prop25a34"] = data["edad25a34"]*100/data["pob_tot"]

In [9]:
import linearmodels as linearmodels
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

## Estimación PooledOLS

In [10]:
exog_vars = ['prop25a34','desocupa','logpibe','pobreza']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.tasa_rcasa, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:             tasa_rcasa   R-squared:                        0.5040
Estimator:                  PooledOLS   R-squared (Between):              0.5469
No. Observations:                 128   R-squared (Within):              -0.0335
Date:                Mon, Jul 20 2020   R-squared (Overall):              0.5040
Time:                        00:44:35   Log-likelihood                   -712.74
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      31.251
Entities:                          32   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(4,123)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             31.251
                            

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:42: RuntimeWarning: invalid value encountered in reduce
  result = getattr(asarray(obj), method)(*args, **kwds)
/srv/conda/envs/notebook/lib/python3.7/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## Estimación con Efectos Aleatorios

In [12]:
from linearmodels.panel import RandomEffects
mod = RandomEffects(data.tasa_rcasa, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:             tasa_rcasa   R-squared:                        0.2133
Estimator:              RandomEffects   R-squared (Between):              0.5319
No. Observations:                 128   R-squared (Within):               0.0108
Date:                Mon, Jul 20 2020   R-squared (Overall):              0.4935
Time:                        00:44:42   Log-likelihood                   -606.99
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      8.3355
Entities:                          32   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(4,123)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             8.3355
                            

## Resultados
### Comparación efectos aleatorios vs Pool OLS

In [19]:
from linearmodels.panel import compare
print(compare({'RE':re_res,'Pooled':pooled_res}))

                 Model Comparison                 
                                   RE       Pooled
--------------------------------------------------
Dep. Variable              tasa_rcasa   tasa_rcasa
Estimator               RandomEffects    PooledOLS
No. Observations                  128          128
Cov. Est.                  Unadjusted   Unadjusted
R-squared                      0.2133       0.5040
R-Squared (Within)             0.0108      -0.0335
R-Squared (Between)            0.5319       0.5469
R-Squared (Overall)            0.4935       0.5040
F-statistic                    8.3355       31.251
P-value (F-stat)               0.0000       0.0000
===================== =============== ============
const                          148.36       121.34
                             (0.4666)     (0.6842)
prop25a34                      37.524       40.646
                             (2.5189)     (4.7447)
desocupa                       12.535       5.2567
                             (2

### Discusion de los resultados

La estimación del modelo arroja los resultados que van en la dirección esperada y  las variables seleccionadas resultan estadísticamente significativas (t>2). La estimación muestra que:

1. A mayor proporción de población entre 25 y 34 años dentro de un estado, mayor será la tasa de robo a casa habitación. El incremento de un 1% en esta proporción de población, manteniendo todo lo demás constante, afectará el robo a casa habitación en 37.5 robos por cada 100 mil habitantes.

2. Entre mayor porcentaje de población económicamente activa desempleada, se espera un incremento en la tasa de robo a casa habitación. En promedio, el aumento en un 1% en la tasa de desocupación implicará un ascenso de 12.5 robos por cada 100 mil habitantes.

3. Las variables relacionadas al PIB estatal y la proporción de población en situación de pobreza, contrario a las variables previas, presentan una relación inversa con los robos a casa habitación. Lo que implicaría que mayor ingreso de las entidades  está asociado a mayores medidas de seguridad. De esta forma, el incremento del PIB estatal en un punto porcentual reduce el número de robos a casa habitación en 48 por cada 100 mil habitantes, en promedio y manteniendo todo lo demás constante.

4. Por su parte, el incremento en la pobreza se asocia a una reducción en el robo a casa habitación, si la pobreza se incrementa en un 1%,  el robo a casa habitación se reduce en 1.9 robos por cada 100 mil habitantes. 

Adicionalmente al modelo de efectos aleatorios, se estimó un modelo de mínimos cuadrados ordinarios con datos agrupados (Pool OLS). La estimación de este modelo se realizó con el fin de  robustecer los resultados del modelo de efectos aleatorios, en términos de los signos y de la magnitud de los parámetros.  Como resultado de ello, el modelo arroja signos iguales en las variables independientes a los presentados en el modelo por efectos aleatorios y parámetros similares en tres de las cuatro variables.

El parámetros de la proporción de población de 25 a 34 años en Pool OLS es equivalente a 40.6 vs 37.5 del modelo de efectos aleatorios. Para la variable PIB estatal este modelo presenta un  parámetro de 46.4 vs 48  del modelo de efectos aleatorios. Por su parte, el parámetro de pobreza asciende a 2.3 vs 1.9 en efectos aleatorios. La desocupación es la variable para la que ambos modelos divergen mayormente en la dimensión del parámetro, 5.2 en Pool OLS y 12.5 en efectos aleatorios. 
Finalmente, el modelo de efectos aleatorios permite explicar el 49.3 % ($R^2 = 0.493$) de la variación de la tasa de robo a casa habitación, a través de las variables mencionadas. Mientras que el modelo Pool OLS  arroja un 50% ($R^2 = 0.504$)


## Anexo
### Robo a transeúnte

En el caso del robo a transeúnte los factores que influyen en este tipo de delito son distintos a los señalados para el robo a casa habitación. Para conocer las variables que afectan este delito tendría que realizarse una revisión de literatura y buscar otras variables. Se incluye el ejercicio como anexo para mostrar que los factores que influyen en este delito difieren de los detectados para robo a casa habitación.

In [ ]:
#  Para poolOLS
exog_vars = ['prop25a34','desocupa','logpibe','pobreza']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.tasa_rcalle, exog)
pooled_calle = mod.fit()
print(pooled_res)

In [ ]:
# Para Efectos aleatorios
from linearmodels.panel import RandomEffects
mod = RandomEffects(data.tasa_rcalle, exog)
re_calle = mod.fit()
#print(re_res)

In [ ]:
#Comparación
from linearmodels.panel import compare
print(compare({'RE':re_calle,'Pooled':pooled_calle}))